In [1]:
from transformers import SegformerModel, SegformerConfig, SegformerForSemanticSegmentation
import torch, json

torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Initializing a SegFormer nvidia/segformer-b0-finetuned-ade-512-512 style configuration
configuration = SegformerConfig()

# Initializing a model from the nvidia/segformer-b0-finetuned-ade-512-512 style configuration
model = SegformerForSemanticSegmentation(configuration).cuda()

# Accessing the model configuration
configuration = model.config

In [5]:

out = model(torch.rand(size=(1, 3, 156, 156), device='cuda'))
out.logits.shape

torch.Size([1, 2, 39, 39])

In [7]:
def measure_vram(ssize):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats("cuda")
    model(torch.rand(size=(1, 3, ssize, ssize), device='cuda'))
    mem_allocation = torch.cuda.max_memory_allocated("cuda")
    mem_allocation_mb = mem_allocation/ 1024**2
    print("Max VRAM:", mem_allocation_mb, "MB")
    return mem_allocation_mb

In [10]:
num_pixel_to_measurements = {}
for mult in [20, 40, 80, 120, 160][::-1]:
    size = mult*16
    num_pixel = size * size
    num_pixel_to_measurements[num_pixel] = []
    for _ in range(3):
        num_pixel_to_measurements[num_pixel].append(measure_vram(size))

json.dump(num_pixel_to_measurements, open("test_segformer.json", "w"), indent=4)


Max VRAM: 21210.04541015625 MB
Max VRAM: 21210.04541015625 MB
Max VRAM: 21210.04541015625 MB
Max VRAM: 7415.83251953125 MB
Max VRAM: 7415.83251953125 MB
Max VRAM: 7415.83251953125 MB
Max VRAM: 2253.03759765625 MB
Max VRAM: 2253.03759765625 MB
Max VRAM: 2253.03759765625 MB
Max VRAM: 1170.44580078125 MB
Max VRAM: 1170.44580078125 MB
Max VRAM: 1170.44580078125 MB
Max VRAM: 991.71728515625 MB
Max VRAM: 991.71728515625 MB
Max VRAM: 991.71728515625 MB
